In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

### Carregando o conjunto de dados para regressão.

+ Vamos usar o conjunto de dados habitacional da Califórnia e criar um regressor com uma rede neural.

+ Depois de carregar os dados, dividimos em um conjunto de treinamento, um conjunto de validação e um conjunto de teste, e padronizamos todos os atributos.

In [3]:
# Baixa a base de dados.
housing = fetch_california_housing()

# Divide o conjunto total de exemplos em conjuntos de treinamento e teste.
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)

# Divide o conjunto de treinamento em conjuntos de treinamento (menor) e validação.
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

# Aplica padronização às matrizes de atributos.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Criando e compilando o modelo.

+ Neste exemplos usaremos a API sequencial, mas o método `save()` funciona também para a API funcional.

In [4]:
model = keras.models.Sequential(
    [
        keras.layers.Input(shape=[8]),
        keras.layers.Dense(30, activation="relu", name='hidden1'),
        keras.layers.Dense(30, activation="relu", name='hidden2'),
        keras.layers.Dense(1, name='output')
    ]
)

model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

### Treinando o modelo.

In [5]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    validation_data=(X_valid, y_valid)
                   )

Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 2.6836 - val_loss: 1.9253
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.8198 - val_loss: 0.6977
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7034 - val_loss: 0.6032
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6609 - val_loss: 0.5869
Epoch 5/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6294 - val_loss: 0.5665
Epoch 6/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6034 - val_loss: 0.5448
Epoch 7/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5812 - val_loss: 0.5240
Epoch 8/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5620 - val_loss: 0.5052
Epoch 9/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5450 - val_loss: 0.4888
Epoch 10/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5300 - val_loss: 0.4732


### Salvando o modelo.

In [6]:
model.save("./my_keras_model.keras")

### Carregando o modelo salvo.

In [7]:
loaded_model = keras.models.load_model("./my_keras_model.keras")

### Realizando predições com o modelo salvo.

In [8]:
X_new = X_test[:3]

y_pred = loaded_model.predict(X_new)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


In [16]:
for i in range(len(y_pred)):
    print('Actual: %1.3f - Predicted: %1.3f' % (y_test[i], y_pred[i,0]))

Actual: 0.477 - Predicted: 0.667
Actual: 0.458 - Predicted: 1.668
Actual: 5.000 - Predicted: 3.141
